# BERTモデルによるテキスト分類

## BERTの利用方法
wikipediaからダウンロードした40文でBERTモデルをfine-tuningし、テキストを分類するモデルを作成、ホスティングします。
BERTとは汎用言語モデルと呼ばれており、Wikipediaのような巨大なコーパスであらかじめ学習済みのモデルです。
学習済みのモデルを利用して、質問応答、文章生成、テキスト分類などのタスクにfine-tuningして利用することができます。

TensorFlowに関しては、こちらに多言語用の学習済みBERTモデルが提供されています。
- GitHubのページ: https://github.com/google-research/bert
- モデルへのリンク: https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip

このモデルを読みこんで分類用にfine-tuningするコードを `src`フォルダに入れています。そのなかの`entry.py`が学習とホスティングを行うためのコードで、BERTを扱うためのソースコードは`src/bert`に入れています。それぞれgithubの以下のコードを流用しています。
- 学習のコード, https://github.com/google-research/bert/blob/master/run_classifier.py
- bertフォルダのコード, https://github.com/google-research/bert

なおTensorFlow以外にも[GluonNLP](https://gluon-nlp.mxnet.io/)が高レベルなラッパーを提供しており、非常に短いコードで学習済みBERTモデルからのfine-tuningを行うことができます。

## 学習用データのS3へのアップロード
`upload_data`関数を利用して、wikipediaの各ページ(うどん、すし、ラーメン、カレー）の計40文をアップロードします。

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

train_text = sagemaker_session.upload_data(path = './corpus_from_wiki.txt')

## セットアップ

In [ ]:

from sagemaker.tensorflow import TensorFlow

bert_estimator = TensorFlow(entry_point='entry.py',
                             role=role,
                             source_dir ="./src",
                             train_instance_count=1,
                             train_instance_type='ml.c5.2xlarge',
                             framework_version='1.12',
                             py_version = 'py3')


## 学習

In [ ]:
bert_estimator.fit(train_text)

## デプロイ

In [ ]:
from sagemaker.tensorflow.model import TensorFlowModel
bert_model = TensorFlowModel(bert_estimator.model_data, role = role, entry_point = 'entry.py', source_dir ="./src", framework_version='1.12')
predictor = bert_model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

## 推論

In [ ]:
import numpy as np

labels = ['うどん', '寿司', 'ラーメン', 'カレー']

query = '文章をここにいれてください。'
result = predictor.predict({"instances": query})

label_index = np.argmax(result['outputs']['probabilities']['float_val'])
print("クエリ: 「{}」".format(query))
print("あなたの文章は {} っぽいです。".format(labels[label_index]))
print()
print(result)

## エンドポイントの削除

In [ ]:
predictor.delete_endpoint()